# ALIGNED project: FAST GSA tutorial

**Aligning Life Cycle Assessment methods and bio-based sectors for improved environmental performance**

[http://www.alignedproject.eu/](http://www.alignedproject.eu/)

_Horizon Europe grant agreement N° 101059430. Views and opinions expressed are however those of the author(s) only and do not necessarily reflect those of the European Union or the European Research Executive Agency._ 


## WP1 Shared modelling framework and learnings
### Task 1.4 Framework for interpreting uncertainty

#### Deliverable 1.2 Description of scientific methods

#### Tutorial for performing Global Sensitivity Analysis using the FAST method

#### Massimo Pizzol, Aalborg University (AAU), 2024

This notebook show how to perform a simple Global Sensitivity Analysis (GSA) applying the FAST, a variance-based sensitivity analysis method, for an example product system of a biobased product.

Credits: special thanks to [Pierre Jouannais](https://www.linkedin.com/in/pierre-jouannais-27736023b/) for preparing an early version of this tutorial.

_Technical note: the SALib library is needed for this tutorial._

In [1]:
# importing packages
import brightway2 as bw
import pandas as pd
import numpy as np
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
# open a project with ecoinvent v.3.9.1 consequential system model
bw.projects.set_current('advlca23')

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [3]:
# Import the dummy product system

# import data from csv
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb = mydata.copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [4]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bw.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

Writing activities to SQLite3 database:
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/14/2024 21:00:36
  Finished: 03/14/2024 21:00:36
  Total time elapsed: 00:00:00
  CPU %: 50.30
  Memory %: 1.25


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [5]:
# check what foreground activities are included
for act in bw.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df


In [6]:
# More info 
myact = bw.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'process',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'}

In [7]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Notes': 'Input of manufacturing',
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [8]:
# calculation of static LCA score
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
myact = bw.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bw.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

121.67148851736351


### Now perform sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input **using a specific and efficient sampling design** (the **"problem"** below)
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) **A sensitivity index is calculated** using model input values (the "problem") and output values.

#### Step 1
Obtain a sample of values for each parameter.


This is a  pseudo-random sample obtained using a sampling method from the SALib package on the problem previously defined.

it is defined as a __FAST__ sample ("Fourier Amplitude Sensitivity Analysis"), another alternative to Sobol method.

Sobol and FAST sampling methods use different pseudo-random patterns to cover the space of parameters, and different calculations for the sensitivity indices.

Besides these differences, the (extended) FAST method is, just like the Sobol one, a variance-based method for sensitivity analysis. It provides the same information as the Sobol method but with higher computational efficiency.

For further information and comparison see: 

_Saltelli, A.; Tarantola, S.; Chan, K. P. S. A Quantitative Model-Independent Method for Global Sensitivity Analysis of Model Output. Technometrics 1999, 41 (1), 39–56._ https://doi.org/10.1080/00401706.1999.10485594.


In [9]:
# need to import the SALib library and relaive methods for sensitvity analysis
# do 'pip install SALib' in the brightway environment

from SALib.sample import saltelli
from SALib.sample import fast_sampler
from SALib.analyze import sobol
from SALib.analyze import fast


Define the "problem" for the GSA analysis as indicated in the SALib library.
Uniform distributions for the uncertainty of for each input parameter are here assumed.

In [10]:
problem = { 'num_vars': 4, # number of variables
            'names': ['par1', 'par2', 'par3', 'par4'], # names of variables, same as parameters
            'bounds': [[-1.25, -0.75], # careful here to what is the lower bound with negative values...
                       [0.1, 0.9],  
                       [20, 60],
                       [0.75, 1.25]] ,
           'dists':["unif","unif","unif","unif"] } # all uniform distributions

In [11]:
# FAST sampler creates N*P parameters sets with N=sample size, P = number of parameters
param_values_FAST = fast_sampler.sample(problem, 200)
print(param_values_FAST.shape)

param_values_FAST

(800, 4)


array([[-0.7648972 ,  0.87616447, 58.80822373,  1.2351028 ],
       [-0.8848972 ,  0.86816447, 58.00822373,  1.2201028 ],
       [-1.0048972 ,  0.86016447, 57.20822373,  1.2051028 ],
       ...,
       [-1.18287965,  0.23139256, 27.76962815,  1.16212035],
       [-1.18787965,  0.21539256, 26.56962815,  1.04212035],
       [-1.19287965,  0.19939256, 25.36962815,  0.92212035]])

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices, taking the data directly from the BW database that we just created.

In [12]:

# In biomass growth, value of CO2 uptake
par1 = list(bw.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bw.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bw.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bw.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = len(param_values_FAST)
param_samples = [(par1['output'],par1['input'], [i[0] for i in param_values_FAST]),
                 (par2['output'],par2['input'], [i[1] for i in param_values_FAST]),
                 (par3['output'],par3['input'], [i[2] for i in param_values_FAST]),
                 (par4['output'],par4['input'], [i[3] for i in param_values_FAST])]

#### Step 2

Iteration through all parameter samples. New LCA scores are calculated for each combination of values.

With 800 iterations, **this will take some time**

In [13]:
# Implementing the loop

GSA_value_results = []


for i in range(0,len(param_values_FAST)): 
    for s in param_samples:

        if s[1][0] == "biosphere3":
            col = LCA.activity_dict[s[0]] # find column index of A matrix for the activity
            row = LCA.biosphere_dict[s[1]] # find row index of B matrix for the exchange
            LCA.biosphere_matrix[row,col] = s[2][i] # substitute the value
        
        else:
            col = LCA.activity_dict[s[0]] # find column index of A matrix for the activity
            row = LCA.activity_dict[s[1]] # find row index of B matrix for the exchange
            LCA.technosphere_matrix[row,col] = -s[2][i] # substitute the value
        
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    #print(LCA.score)
    GSA_value_results.append(LCA.score)

#### Step 3

Now we have both the input and output values and we can feed the problem and the results to the Sobol function to obtain the FAST indices.

First we look at the parameters and the result



In [14]:
# Organize data first to give a look at what we have done: a sample of input values and a corresponding output value
fast_data = pd.DataFrame([i[2] for i in param_samples], index = ['par0','par1', 'par2', 'par3']).T
fast_data['GWI'] = GSA_value_results
fast_data.head(10) # show only the first ten samples


,par0,par1,par2,par3,GWI
0,-0.764897,0.876164,58.808224,1.235103,153.236427
1,-0.884897,0.868164,58.008224,1.220103,147.766029
2,-1.004897,0.860164,57.208224,1.205103,142.391894
3,-1.124897,0.852164,56.408224,1.190103,137.114021
4,-1.244897,0.844164,55.608224,1.175103,131.932411
5,-1.135103,0.836164,54.808224,1.160103,133.144374
6,-1.015103,0.828164,54.008224,1.145103,134.544356
7,-0.895103,0.820164,53.208224,1.130103,135.848601
8,-0.775103,0.812164,52.408224,1.115103,137.057108
9,-0.844897,0.804164,51.608224,1.100103,133.272401


Now we calculate the SI index

In [15]:
si = fast.analyze(problem, np.array(GSA_value_results), print_to_console=True) # must use np.array

            S1        ST   S1_conf   ST_conf
par1  0.025311  0.028922  0.071829  0.096111
par2  0.000112  0.001040  0.078198  0.078956
par3  0.805798  0.810176  0.078177  0.094127
par4  0.155082  0.157187  0.075938  0.089059


In this specific case "par3"  is the parameter to which the results are most sensitive to, this because it has the highest value for "S1" which is the first order effect.

This is determined with relativelygood confidence as the error (value of "S1_conf") is small compared to the coefficient.

Note also that "par3" is also the parameter withi highest sensitivity in the total model, i.e. when in combinatino with other parameters, as observed by the highest value of "ST".

In [16]:
# In this specific case "par3" is the parameter to which the results ar emost sensitive to, and values have high confidente (small error)
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>